<center><h1> Exponential Decay

In [ ]:
from mud_problem import generate_sensors_ode
from mud_problem import makeDecayModel
from experiments import mud_problem
from mud.util import std_from_equipment
import numpy as np

## Generate Sensors

In [ ]:
t_min, t_max = 1, 3

In [ ]:
sensors = generate_sensors_ode(measurement_hertz=100,
                               start_time=t_min,
                               end_time=t_max)

## Time / Measurement Snapshots

In [ ]:
# what proportion of the experiment will we simulate solutions for?
time_ratios = [0.01, 0.05, 0.1, 0.25, 0.5, 1]
measurements = [ int(np.floor(len(sensors)*r)) for r in time_ratios ]
print(f"Measurements for which we will solve the problem: {measurements}")

In [ ]:
lam_true = [0.5]
model = makeDecayModel(sensors, lam_true)

In [ ]:
qoi_true = model() # no args evaluates true param

## Sample $\Lambda$ and Map to $\mathcal{D}$

In [ ]:
domain=[[0,1]]
lam = np.random.rand(int(1E4)).reshape(-1,1)
qoi = model(lam)

In [ ]:
lam.shape, qoi.shape

Define wrapper which we use to solve the problem, which takes arguments and passes them along to `bet==2.3.0`.
This wrapper ensures our model is unseen by the solution method.
It only gets input/output data, and must arrive at the most likely estimate of truth from the given set of model evaluation.

In [ ]:
# this relies on BET code that was not ported to version 3.0
# I plan to upgrade it in the future, but LOOK HOW CLEAN IT IS
mud_problem??

In [ ]:
sigma = std_from_equipment(tolerance=0.1, probability=0.99)
def mud_wrapper(num_obs, sd=sigma):
    return mud_problem(domain=domain, lam=lam, qoi=qoi, sd=sd, qoi_true=qoi_true, num_obs=num_obs)


## Example Usage

In [ ]:
disc = mud_wrapper(20)

In [ ]:
disc.mud_point()

Evaluate estimate of updated density (i.e. free of model evals) to see if we can arrive at an even better estimate.

## Run Experiment

In [ ]:
from experiments import experiment_measurements, extract_statistics
from plotting import plot_surface, plot_experiment_measurements, plot_experiment_equipment, log_linear_regression, plot_decay_solution

In [ ]:
import pickle
sol_fname = 'decaySols.pkl'

In [ ]:
%%time
load = True
if not load:

    num_trials = 20
    seed = 21
    print("Increasing Measurements Quantity Study")
    experiments, solutions = experiment_measurements(
                                             num_measurements=measurements,
                                             sd=sigma,
                                             num_trials=num_trials,
                                             seed=seed,
                                             fun=mud_wrapper)

    pickle.dump(solutions, open(sol_fname,'wb'))
else:
    solutions = pickle.load(open(sol_fname,'rb'))
    num_trials = len(list(solutions.values())[0])
    measurements = list(solutions.keys())

## Extract Statistics

In [ ]:
means, variances = extract_statistics(solutions, lam_true)
regression_mean, slope_mean = log_linear_regression(time_ratios, means)
regression_vars, slope_vars = log_linear_regression(time_ratios, variances)


## Plotting

In [ ]:
fsize = 32
linewidth = 5

In [ ]:
plot_decay_solution?

In [ ]:
subset_sols = {n: solutions[n] for n in [2,20,200]}
plot_decay_solution(subset_sols, makeDecayModel, fsize=fsize,
                    end_time=t_max, lam_true=lam_true, qoi_true=qoi_true,
                    sigma=sigma, time_vector=sensors, prefix='ode/ode')

In [ ]:
_rm = (regression_mean, slope_mean, regression_vars, slope_vars, means, variances)

In [ ]:
# plot_experiment_measurements??

In [ ]:
plot_experiment_measurements(measurements, [('Mean Absolute Error', _, _rm, _)],
                             'ode/ode', fsize, linewidth,
                             test=False)